In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IcebergSparkWithMinIO") \
    .config("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg_catalog.type", "rest") \
    .config("spark.sql.catalog.iceberg_catalog.uri", "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.iceberg_catalog.warehouse", "s3://warehouse/") \
    .config("spark.sql.catalog.iceberg_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg_catalog.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.catalog.iceberg_catalog.s3.access-key-id", "admin") \
    .config("spark.sql.catalog.iceberg_catalog.s3.secret-access-key", "password") \
    .config("spark.sql.catalog.iceberg_catalog.s3.path-style-access", "true") \
    .getOrCreate()

In [ ]:
spark.sql("SHOW DATABASES IN iceberg_catalog").show()

In [ ]:
spark.conf.get("spark.sql.catalog.iceberg_catalog")

In [ ]:
spark.conf.get("spark.sql.catalog.iceberg_catalog.warehouse")

In [ ]:
spark.conf.get("spark.sql.catalog.iceberg_catalog.type")

In [ ]:
spark.conf.get("spark.sql.catalog.iceberg_catalog.uri")

In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg_catalog.default")

In [ ]:
spark.sql("""
    CREATE TABLE iceberg_catalog.default.my_table (
        id BIGINT,
        name STRING
    ) USING iceberg
""")

In [ ]:
spark.sql("INSERT INTO iceberg_catalog.default.my_table VALUES (1, 'Alice'), (2, 'Bob')")
df = spark.sql("SELECT * FROM iceberg_catalog.default.my_table")
df.show()

In [ ]:
spark.sql("SELECT * FROM iceberg_catalog.default.my_table.snapshots").show()

In [ ]:
spark.sql("UPDATE iceberg_catalog.default.my_table VALUES SET name = 'David' WHERE id = 1")
df = spark.sql("SELECT * FROM iceberg_catalog.default.my_table")
df.show()

In [ ]:
spark.sql("SELECT * FROM iceberg_catalog.default.my_table.snapshots").show()

In [ ]:
spark.sql("INSERT INTO iceberg_catalog.default.my_table VALUES (3, 'Brian'), (4, 'Joe')")
df = spark.sql("SELECT * FROM iceberg_catalog.default.my_table")
df.show()

In [ ]:
spark.sql("SELECT * FROM iceberg_catalog.default.my_table.snapshots").show()

In [ ]:
spark.sql("""
CALL iceberg_catalog.system.rollback_to_snapshot('default.my_table', 8784649690137705673)
""").show()

In [ ]:
spark.sql("""
CALL iceberg_catalog.system.rollback_to_timestamp('default.my_table', TIMESTAMP '2026-01-05 03:29:53.542')
""").show()

In [ ]:
spark.sql("""
CALL iceberg_catalog.system.set_current_snapshot('default.my_table', 5255231086484106368)
""").show()

In [ ]:
spark.sql("SELECT * FROM iceberg_catalog.default.my_table").show()

In [ ]:
spark.sql("""
CALL iceberg_catalog.system.set_current_snapshot('default.my_table', 5255231086484106368)
""").show()

In [ ]:
spark.sql("SELECT * FROM iceberg_catalog.default.my_table").show()

In [ ]:
spark.sql("""
SELECT * FROM iceberg_catalog.default.my_table.history
""").show(truncate=False)